## 1.a) Extraction

In [ ]:
#Importing libraries

import pandas as pd 
import numpy as np
import seaborn as sns 
import matplotlib.pyplot as plt 
import requests

print("Libraries imported successfully......")

The link to all these APIs is: https://www.game-change.co.uk/2023/02/10/a-complete-guide-to-the-fantasy-premier-league-fpl-api/

Descriptions of each dataframe are in the notes.

### i. Player data Current Season


In [ ]:
### Function to return player url 
def format_player_url(player_id):
    # Define the base URL without curly brackets
    base_url = 'https://fantasy.premierleague.com/api/element-summary/{}/'
    
    # Replace the placeholder '{}' with the actual player_id
    formatted_url = base_url.format(player_id)
    
    return formatted_url

# Example usage:
player_id = 308  # Replace this with the actual player ID
formatted_url = format_player_url(player_id)
print(formatted_url)


In [ ]:
#This cell imports individual player information
r = requests.get(format_player_url(308))
player_json = r.json()


player_fixtures_df = pd.DataFrame(player_json['fixtures'])
player_history_df = pd.DataFrame(player_json['history'])
player_history_past_df = pd.DataFrame(player_json['history_past'])


In [ ]:
### This cell imports data from specified gameweek of the current season as long as the gameweek has ended or is live.
gameweek_url = 'https://fantasy.premierleague.com/api/event/2/live/'
r = requests.get(gameweek_url)
gameweek_json = r.json()

gameweek_df = pd.DataFrame(gameweek_json['elements'])


### ii. Team data

In [ ]:
# Imported full data from last seasons

url = 'https://fantasy.premierleague.com/api/bootstrap-static/'
r = requests.get(url)
json = r.json()


#Convert to df

elements_df = pd.DataFrame(json['elements'])
elements_types_df = pd.DataFrame(json['element_types'])
teams_df = pd.DataFrame(json['teams'])

### iii. Full Gameweek data 

In [ ]:
#This cell imports Full gameweek information.
fixture_url = 'https://fantasy.premierleague.com/api/fixtures/?event=2'
r = requests.get(fixture_url)
fixture_json = r.json()


fixture_df = pd.DataFrame(fixture_json)



## 1.b) Tranformation

### i. Add fixture data to the teams data

#### i.i) Get fixture id

In [ ]:
teams_df = teams_df.rename(columns={'name' : 'team_name', 'id':'teams_id', 'short_name' : 'team_short_name'})

In [ ]:
fixture_df = fixture_df.rename(columns={'id':'fixture_id'})

In [ ]:
def get_fixture_id(team_id):
    for idx, row in fixture_df.iterrows():
        if team_id == row['team_a'] or team_id == row['team_h']:
            return row['fixture_id']
    return None  # Return None if no matching fixture is found

# Assuming you want to calculate and assign fixture IDs for all teams in teams_df
teams_df['fixture_id'] = teams_df['teams_id'].apply(get_fixture_id)


#### i.ii) Get fixture difficulty

In [ ]:
def get_fixture_difficulty(team_id, fixture_id):
    for idx, row in fixture_df.iterrows():
        if fixture_id == row['fixture_id']:
            if team_id == row['team_a']:
                return row['team_a_difficulty']

            elif team_id == row['team_h']:
                return row['team_h_difficulty']
    return None  # Return None if no matching fixture is found

# Assuming you want to calculate and assign fixture difficulties for all teams in teams_df
teams_df['fixture_difficulty'] = teams_df.apply(lambda row: get_fixture_difficulty(row['teams_id'], row['fixture_id']), axis=1)


## 1.b) Loading


In [ ]:
player_history_df